# TorchArrow in 10 minutes

TorchArrow is a Python DataFrame library built on the Apache Arrow columnar memory format and leveraging the Velox vectorized engine for loading, filtering, mapping, joining, aggregating, and otherwise manipulating tabular data on CPUs.

TorchArrow allows mostly zero copy interop with Numpy, Pandas, PyArrow, CuDf and of coarse PyTorch.
In fact it is the integration with PyTorch which has trigered the development of TorchArrow. 
So TorchArrow understands Tensors natively.  

(Remark. In case the following looks familar, it is with gratitude that portions of this tutorial were borrowed and adapted from the 10 Minutes to Pandas (and CuDF) tutorial.)



In [1]:

import pandas as pd
import numpy as np
import pyarrow as pa

The TorchArrow library consists of 3 parts: 

  * *DTypes* define *Schema*, *Fields*, primitive and composite *Types*. 
  * *Columns* defines sequences of strongly typed data with vectorized operations.
  * *Dataframes*  are sequences of named and typed columns of same length with relational operations.  

Let's get started...

In [2]:
import torcharrow as ta

## Constructing data: Columns

### From Pandas to TorchArrow
To start let's create a Panda series and a TorchArrow column and compare them:

In [3]:
pd.Series([1,2,None,4])

0    1.0
1    2.0
2    NaN
3    4.0
dtype: float64

In Pandas each Series has an index, here depicted as the first column. Note also that the inferred type is float and not int, since in Pandas None implictly promotes an int list to a float series.

TorchArrow has a much more precise type system:

In [4]:
s = ta.Column([1,2,None,4])
s

0  1
1  2
2  None
3  4
dtype: Int64(nullable=True), length: 4, null_count: 1

TorchArrow infers that that the type is `Int64(nullable=True)` which required that the vectors is represented internally via two arrays, its data and validity bit mask (the current implementation uses byte for each bit). We can make the internal representation explicit by calling


In [5]:
s.show_details()

  data    validity
------  ----------
     1           1
     2           1
     0           0
     4           1
dtype: Int64(nullable=True), count: 4, null_count: 1, offset: 0

Of course we can always get lots of more informataion from a column (its length, memory_usage etc.):

In [6]:
(len(s), s.memory_usage())

(4, 36)

TorchArrow supports (almost all of Arrow types), including arbitrarily nested structs, maps, lists, and fixed size lists. Here is a column of a list of strings.

In [7]:
sf = ta.Column([ ["hello", "world"], ["how", "are", "you"] ], ta.List_(ta.string))
sf

0  ['hello', 'world']
1  ['how', 'are', 'you']
dtype: List_(string), length: 2, null_count: 0

And here is column of average climate data, one map per contintent, with city as key and yearly average min and max temperature:


In [8]:
mf = ta.Column([ 
    {'helsinki': [-1.3, 21.5], 'moskow': [-4.0,24.3]}, 
    {'algiers':[11.2, 25,2], 'kinshasa':[22.2,26.8]}
    ])
mf

0  {'helsinki': [-1.3, 21.5], 'moskow': [-4.0, 24.3]}
1  {'algiers': [11.2, 25.0, 2.0], 'kinshasa': [22.2, 26.8]}
dtype: Map(string, List_(float64)), length: 2, null_count: 0

### Builders

Columns are append only. Use the usual `append` and `extend` funcions to grow them.

In [9]:
sf.append(["I", "am", "fine"])
sf

0  ['hello', 'world']
1  ['how', 'are', 'you']
2  ['I', 'am', 'fine']
dtype: List_(string), length: 3, null_count: 0

TorchArrow's mutability model supports mutiple readers, a single writer and copy on write. Consult the notebook *torcharrow_mutability.ipynb* for more details on this model.


## Constructing data: Dataframes

Now let's focus on Dataframes. A Dataframe is just a set of named and strongly typed columns of equal length:

In [10]:
df = ta.DataFrame({'a': list(range(7)),
                     'b': list(reversed(range(7))),
                     'c': list(range(7))
                    })
df

  index    a    b    c
-------  ---  ---  ---
      0    0    6    0
      1    1    5    1
      2    2    4    2
      3    3    3    3
      4    4    2    4
      5    5    1    5
      6    6    0    6
dtype: Struct([Field('a', int64), Field('b', int64), Field('c', int64)]), count: 7, null_count: 0

Dataframes allows *a single writer* to extend a dataframe with additional columns, provided they have the same length. 

In [11]:
df['d'] = ta.Column(list(range(99, 99+7)))
df

  index    a    b    c    d
-------  ---  ---  ---  ---
      0    0    6    0   99
      1    1    5    1  100
      2    2    4    2  101
      3    3    3    3  102
      4    4    2    4  103
      5    5    1    5  104
      6    6    0    6  105
dtype: Struct([Field('a', int64), Field('b', int64), Field('c', int64), Field('d', int64)]), count: 7, null_count: 0

Like columns, dataframes can be nested. Here is a Dataframe having sub-dataframes. 


In [12]:

df_inner = ta.DataFrame({'b1': [11, 22, 33], 'b2':[111,222,333]})
df_outer = ta.DataFrame({'a': [1, 2, 3], 'b':df_inner})
df_outer

  index    a  b
-------  ---  ---------
      0    1  (11, 111)
      1    2  (22, 222)
      2    3  (33, 333)
dtype: Struct([Field('a', int64), Field('b', Struct([Field('b1', int64), Field('b2', int64)]))]), count: 3, null_count: 0

Dataframes follow the same mutability model as columns. That is I can not only add columns, but I can also add rows:


In [13]:
df_outer.append((4,(44,444)))
df_outer

  index    a  b
-------  ---  ---------
      0    1  (11, 111)
      1    2  (22, 222)
      2    3  (33, 333)
      3    4  (44, 444)
dtype: Struct([Field('a', int64), Field('b', Struct([Field('b1', int64), Field('b2', int64)]))]), count: 4, null_count: 0

## Interop

Take a Pandas dataframe and move it zero copy (if possible) to TorchArrow.

In [14]:

pdf = pd.DataFrame({'a': [0, 1, 2, 3],'b': [0.1, 0.2, None, 0.3]})
gdf = ta.from_pandas_dataframe(pdf)
gdf

  index    a    b
-------  ---  ---
      0    0  0.1
      1    1  0.2
      2    2
      3    3  0.3
dtype: Struct([Field('a', int64), Field('b', Float64(nullable=True))]), count: 4, null_count: 0

And bring it back to Pandas

In [15]:
gdf.to_pandas()

,a,b
0,0,0.1
1,1,0.2
2,2,None
3,3,0.3


The same works for arrow, too. 

In [16]:
ta.from_arrow_table(pa.table({'a': [0, 1, 2, 3],'b': [0.1, 0.2, None, 0.3]})).to_arrow()

pyarrow.Table
a: int64
b: double

## Viewing (sorted) data

Take the (head of) the top n rows

In [17]:
df.head(2)

  index    a    b    c    d
-------  ---  ---  ---  ---
      0    0    6    0   99
      1    1    5    1  100
dtype: Struct([Field('a', int64), Field('b', int64), Field('c', int64), Field('d', int64)]), count: 2, null_count: 0

Or return the last n rows

In [18]:
df.tail(1)


  index    a    b    c    d
-------  ---  ---  ---  ---
      0    6    0    6  105
dtype: Struct([Field('a', int64), Field('b', int64), Field('c', int64), Field('d', int64)]), count: 1, null_count: 0

Sort values

In [19]:
df.sort_values(by='b').head(2)

  index    a    b    c    d
-------  ---  ---  ---  ---
      0    6    0    6  105
      1    5    1    5  104
dtype: Struct([Field('a', int64), Field('b', int64), Field('c', int64), Field('d', int64)]), count: 2, null_count: 0

## Selection
Projection a single column

In [20]:
df['a']

0  0
1  1
2  2
3  3
4  4
5  5
6  6
dtype: int64, length: 7, null_count: 0

Selection by row position returns a row.

In [21]:
df[1]

(1, 5, 1, 100)

Selecting a slice keeps the type alive.


In [22]:
df.slice(2,3)

  index    a    b    c    d
-------  ---  ---  ---  ---
      0    2    4    2  101
dtype: Struct([Field('a', int64), Field('b', int64), Field('c', int64), Field('d', int64)]), count: 1, null_count: 0

In fact torcharrow supports all of Python's integer slice notation. 

In [23]:
df[2:6:2]

  index    a    b    c    d
-------  ---  ---  ---  ---
      0    2    4    2  101
      1    4    2    4  103
dtype: Struct([Field('a', int64), Field('b', int64), Field('c', int64), Field('d', int64)]), count: 2, null_count: 0

In adition you can slice by strings, i.e. columns.

In [24]:
df['c':]

  index    c    d
-------  ---  ---
      0    0   99
      1    1  100
      2    2  101
      3    3  102
      4    4  103
      5    5  104
      6    6  105
dtype: Struct([Field('c', int64), Field('d', int64)]), count: 7, null_count: 0

Torcharrow follows the normal Python semantics for slices: that is a slice intervals are closed on the left and open on the right open, or said differently the left most element is included, the right most element is excluded.

Selection by condition is written with a boolean condition (see operators below).

In [25]:
df[df['a'] > 4]

  index    a    b    c    d
-------  ---  ---  ---  ---
      0    5    1    5  104
      1    6    0    6  105
dtype: Struct([Field('a', int64), Field('b', int64), Field('c', int64), Field('d', int64)]), count: 2, null_count: 0

Selection by methods like isin

In [26]:
df[df['a'].isin([5])]

  index    a    b    c    d
-------  ---  ---  ---  ---
      0    5    1    5  104
dtype: Struct([Field('a', int64), Field('b', int64), Field('c', int64), Field('d', int64)]), count: 1, null_count: 0

## Missing data
 Missing data can be filled in via the `fillna` method 

In [27]:
t = s.fillna(999)
t

0    1
1    2
2  999
3    4
dtype: int64, length: 4, null_count: 0

Alternatively data that has null data can be dropped:

In [28]:
s.dropna()

0  1
1  2
2  4
dtype: int64, length: 3, null_count: 0

## Operators
Columns and dataframes support all of Python's usual operators, like  ==,!=,<=,<,>,>= for eqaulity and comparison,  +,-,*,,/.//,** for performing arithmetic and &,|,~ for conjunction, disjunction and negation. 

The semantics of each operator is given by lifting their scalar operation to vectors and dataframes. So given for instance a scalar comparison operator, in toracharrow a scalar can be compared to each item in a column, two columns can be compared pointwise, a column can be compared to each column of a dataframe, and two dataframes can be compared by comparing each of their respective columns. 

Here are some example expressions:

In [29]:
u = ta.Column(list(range(5)))
v = -u
w = v+1
v*w

0   0
1   0
2   2
3   6
4  12
dtype: int64, length: 5, null_count: 0

In [30]:
uv = ta.DataFrame({'a': u, 'b': v})
uu = ta.DataFrame({'a': u, 'b': u})
(uv==uu)

  index  a     b
-------  ----  -----
      0  True  True
      1  True  False
      2  True  False
      3  True  False
      4  True  False
dtype: Struct([Field('a', boolean), Field('b', boolean)]), count: 5, null_count: 0

## Numerical columns and descriptive statistics
Numerical columns also support lifted operations, for abs, ceil floor, round. Even more excited might be to use their aggregation operators like count, sum, prod, min, max, or descriptive statistics like std, mean, median, and mode. Here is an example ensamble:


In [31]:
(t.min(), t.max(), t.sum(), t.mean())

(1, 999, 1006, 251.5)

The `describe` method puts this nicely together: 

In [32]:
t.describe()

  index  statistic      value
-------  -----------  -------
      0  count          4
      1  mean         251.5
      2  std          498.335
      3  min            1
      4  25%            1.75
      5  50%            3
      6  75%          252.75
      7  max          999
dtype: Struct([Field('statistic', string), Field('value', float64)]), count: 8, null_count: 0

Sum, prod, min and max are also available as accumulating operators. 

## String, list and map methods
Torcharrow provides all of Python's string, list and map processing methods, just lifted to work over columns. Like in Pandas they are all accessible via the `str`, `list` and `map` property, respectivly:

In [33]:
s = ta.Column(['what a wonderful world!', 'really?'])
s.str.capitalize()

0  'What a wonderful world!'
1  'Really?'
dtype: string, length: 2, null_count: 0

Split gets an extra parameter called expand to specify whether split should return a list of strings (expand=False) or return list of a columns (expand=True). Join is the inverse. Here is an example:

In [34]:
ss= s.str.split(sep=' ')
ss

0  ['what', 'a', 'wonderful', 'world!']
1  ['really?']
dtype: List_(string), length: 2, null_count: 0

In [35]:
ss.list.join(sep='-')

0  'what-a-wonderful-world!'
1  'really?'
dtype: string, length: 2, null_count: 0

To operate on a list column use the usual pure list operations, like len(gth), get, index and count. In addition lists provide filter, map, flatmap and reduce operators. 

In [36]:
ss.list.map(len, dtype=ta.List_(ta.int64))

0  [4, 1, 9, 6]
1  [7]
dtype: List_(int64), length: 2, null_count: 0

Column of type map provide the usual map operations like len(gth), get, keys and values. Keys and values both return a list column. Key and value columns can be reassembled by calling mapsto.

## Functional tools:  map, filter, reduce

Column and dataframe piplines support functional compositions as well. We start with column oriented operations: `map` maps values of a column according to input correspondence. The input correspondance can be given as a mapping or as a function.


In [37]:
def add_ten(num):
    return num + 10

ta.Column([1,2,None,4]).map(add_ten) == ta.Column([1,2,None,4]).map({i:i+10 for i in range(7)})

0  True
1  True
2  None
3  True
dtype: Boolean(nullable=True), count: 4, null_count: 1

Note that all operations working on columns and dataframes ignore null values. By setting the additional parameter `na_action` to 'ignore', null values will be passed to the mapping as well.
 
If a function's argument type differs from its return type, then the reuturn type must be specified.

In [38]:

ta.Column([1,2,3,4]).map(str, dtype=ta.string)

0  '1'
1  '2'
2  '3'
3  '4'
dtype: string, length: 4, null_count: 0

Filter selects rows where a given mask or predicate is True.

In [39]:
ta.Column([1,2,3,4]).filter([True, False, True, False]) == ta.Column([1,2,3,4]).filter(lambda x: x%2==1)

0  True
1  True
dtype: boolean, count: 2, null_count: 0

 Note that torcharrows's `filter` differs from Pandas `filter`. In Pandas `filter` is a column projection. For column projection use `keep`.  
 
 `flatmap` combines `filter` with `map`. For instance, lets double all rows that start with 'I" and rop all others.

In [40]:
def selfish(words):
    return [words, words] if len(words)>=1 and words[0] == "I" else []

sf.flatmap(selfish)

0  ['I', 'am', 'fine']
1  ['I', 'am', 'fine']
dtype: List_(string), length: 2, null_count: 0

Finally `reduce` works exactly as in Python. To compute the product simply use the opertor mul.

In [41]:
import operator
ta.Column([1,2,3,4]).reduce(operator.mul)

24

All of these functional tools also work on dataframes. For dataframes the whole row (or a subset of it) is passed in as a tuple. 

In [42]:
lst = [1,2,3]
hf = ta.DataFrame({'a': lst, 'b': lst})
hf.map(lambda tup: tup[0]+tup[1], dtype = ta.int64)

0  2
1  4
2  6
dtype: int64, length: 3, null_count: 0

But note that functinal tools should only be used as a last resort. In most cases an equivalent vectorized expression can do the same computation faster. For instance the last exprssion can be simply expressed as `hf['a']+hf['b']`.

# THIS STILL NEEDS TO BE UPDATED

## Relational tools: Join and Group-by
 
Performing SQL style joins. Note that the dataframe order is not maintained. (Is not yet implemened)

In [43]:
# df_a = ta.DataFrame()
# df_a['key'] = ['a', 'b', 'c', 'd', 'e']
# df_a['vals_a'] = [float(i + 10) for i in range(5)]

# df_b = ta.DataFrame()
# df_b['key'] = ['a', 'c', 'e']
# df_b['vals_b'] = [float(i+100) for i in range(3)]

# merged = df_a.merge(df_b, on=['key'], how='left')
# merged

### Grouping

Like pandas, torchArrow support the Split-Apply-Combine groupby paradigm. (Is not yet implemened)

### Transpose



In [44]:
# sample = ta.DataFrame({'a': [1, 2, 3], 'b': [4, 5, 6]})
# sample

In [45]:
# sample.transpose() -TODO

## Extending torcharrow with User defined functions

Above we  we covered the most basic usage of a unary UDF. Let's look into more esoteric features here:

**Multiparameter UDFs.** Functions that take more than one argument but not the complete row declare which columns are passed in.   

In [46]:
# df.map(operator.add, incols= ['a','b']) -- TODO

**Multireturn UDFs.**  Functions that return more than one column can be specfied by returning a tuple; providing the  return type is mandatory.


In [47]:
# df.map(divmod,  incols= ['a','b'], dtypes = [int64, int64]]) -- TODO

**Functions with state**. UDFs need sometimes additional precomputed state. We capture the state in an object and use a method as a delegate:
 

In [48]:
# def fib(n):
#     if n == 0:
#         return 0
#     elif n == 1 or n == 2:
#         return 1
#     else:
#         return fib(n-1) + fib(n-2)
    
# class State:
#     def __init__(self, x):
#         self.state = fib(x) 
#     def add_fib(self, x):
#         return self.state+x

# m = State(10)
# ta.Column([1,2,3]).map(m.add_fib)

## Vectorized user defined functions and transforms

Vectorized function leak TorchArrow representation boundaries! So read the following with the big caveat that it can change quickly!

Vectorized functions get *n* strongly typed vectors as input and return *m* vectors as output. Validity handling is optional. The following assumes that all data is valid!

In [49]:

# def conditional_add(x, y, out):
#     for i, (a, e) in enumerate(zip(x, y)):
#         if a > 0:
#             out[i] = a + e
#         else:
#             out[i] = a

This code is perfect for vectorization via Numba. Leveraging Numba will require us to only add some custom attributes. (TODO)

Vectorized functions can be applied using `transform`. We pass a list of data columns and return a typed list of data columns. 

In [50]:
# df = ta.transform(conditional_add, incols= ['a','b'], dtypes = [int64]) -- TODO
# df.head()

If you want to pass the underlying vaidity map in and/or out as well, you have to provide it as  incols and out dtypes respectively. The input and output names are called name.data and name.vaidity repectively. The dtype for a validity map is called nullable. So for the folowing transfor, we pass all data and validity masks and return a validity vector as well. 

In [51]:
# ta.transform(conditional_add_with_mask, incols = ['a.data','a.mask', 'b.data', 'b.mask'], dtype = [int64, nullable]]) -- TODO

Assuming that nulls are handled as bitarrays of 64 bytes each, and that the return must be null if row a's value is > 0, then we can define it like so.

In [52]:
"End of tutorial"

'End of tutorial'

## Extending torcharrow with user defined types
The notebook *torcharrow_user_defined_types* describe how we can modularly extend torcharrow with new types. As example we will use Tensors. In fact all concrete columns follow the same paradigm. That is w ehave to define a file called X_couln, with two classes and add the class to the int and the two factories. 